In [1]:
import tensorflow as tf
import requests
import pandas as pd
import os
import base64
import json

In [2]:
data_dir = "data_clean"
file_name = 'fake_job_postings_clean.csv'

In [3]:
df = pd.read_csv(data_dir + '/' + file_name)

In [4]:
test_fake = df['text'][0]
test_real = df['text'][98]

In [5]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [6]:
def predict(text):
    json_data = prepare_json(text)
    
    endpoint = "http://localhost:8080/v1/models/real-or-fake-jobs-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "Berita Faktual" if prediction > 0.7 else "Berita Palsu"

In [7]:
predict(test_fake)

'Berita Palsu'

In [8]:
predict(test_real)

'Berita Faktual'